# HOW TO INTEGRATE LLM-POWERED CHATBOT WITH A TELEGRAM FOR FREE
(source: https://www.italiens.ee/en/blog/how_to_integrate_llm_powered_chatbot_with_telegram_for_free/)

# Before starting 
What You’ll Need
- A Telegram account
- A Hugging Face account (free)
- Basic Python setup (Python installed, and ideally a virtual environment set up)

## Step 1: Create Your Telegram Bot
1. Open Telegram on your device.
2. Search for BotFather and click on the ☑️ verified account.
3. Type /newbot and follow the instructions:
    - Give your bot a name
    - Create a username that ends with bot
4. BotFather will send you a bot token. Copy this token — you’ll need it soon.

## Step 2: Get a Free LLM API Key from Hugging Face
1. Go to huggingface.co and sign up (free).
2. Click your profile icon → Access Tokens.
3. Click New Token, give it a name (e.g., "LLMBotToken"), and create it with write access.
4. Copy the API token.
5. Choose a free model from the Hugging Face Model Hub. Free models have open license (e.g. MIT, Apache 2.0). In this guide we will use DeepSeek-R1-Distill-Qwen-1.5B.
    - Keep in mind, this is a reasoning model, so it's quite verbose. You can "play" with it directly on the Hugging Face website, or choose another one.
6. Click Deploy → Inference Providers. Here you can choose an Inference Provider to deploy selected model and access sample code snippets for Python, JavaScript, and cURL.

## Step 3: Install Required Libraries
1. Create a new folder for your project and open it in your terminal.
2. You will need to install the following libraries to interact with the Telegram bot API and Hugging Face:
    ```ruby
    pip install python-telegram-bot huggingface_hub
    ```

## Step 4: Prepare the Python Telegram Bot Script
1. Create a file named bot.py.
2. Paste the following code inside the file


## First import libraries
1. telegram / telegram.ext: librería oficial python-telegram-bot.

    - Update: objeto que representa un evento entrante (mensaje, comando, etc.).
    - ApplicationBuilder: constructor de la app del bot (reemplaza al viejo Updater en versiones modernas).
    - CommandHandler: maneja comandos como /start.
    - MessageHandler: maneja mensajes de texto normales.
    - ContextTypes: tipo para el context que acompaña al update (útil para anotar tipos y acceder a data).
    - filters: condiciones para MessageHandler (p. ej. filters.TEXT).

2. InferenceClient de huggingface_hub: cliente para llamar a modelos alojados (aquí lo usan para chat/completions).

In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters
from huggingface_hub import InferenceClient

Hello


## Declarar TOKENS
- TELEGRAM_TOKEN es el Token del bot de Telegram
- HF_API_KEY es el Token de Huggingface como inference provider

In [ ]:
TELEGRAM_TOKEN = 'YOUR_TELEGRAM_BOT_TOKEN'
HF_API_KEY = 'YOUR_HF_API_TOKEN'

## Inicialización del cliente y almacenar conversaciones

La función InferenceClient(...) crea el cliente para llamar al servicio de inferencia. "provider='novita'" es un parametro de proveedor asociado al LLM elegido. 

La variable user_conversations = {} en un dict en memoria que mapea user_id con la lista de mensajes (historial para contexto). Ojo! Esto se borra al reiniciar y puede crecer indefinidamente. 

In [ ]:
client = InferenceClient(
    provider="novita",
    api_key=HF_API_KEY,
)

user_conversations = {}

## Handler del comando start 

Cuando un usuario manda /start, Telegram envía un Update. La función async (asíncrona) recibe ese Update y update.message.reply_text(...) envía texto de vuelta al usuario (aquí el bot da una bienvenida).

In [ ]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Hello! Send me a message and I will reply using AI!')


## Función main y arranque del bot

- ApplicationBuilder().token(...).build(): crea la aplicación del bot con tu token.
- Registra handlers:
    - /start → start
    - Mensajes de texto que no sean comandos → handle_message (la expresión filters.TEXT & (~filters.COMMAND) filtra solo texto normal).
- run_polling() inicia el "polling" — el bot consulta a los servidores de Telegram continuamente por nuevos updates (alternativa: webhook).
- if __name__ == '__main__': main() permite ejecutar el script como programa.

In [ ]:
def main():
    #Crea una instancia de la aplicación del bot que se conecta a los servidores de Telegram, escucha los mensajes entrantes, 
    # llama autométicamente a las funciones que registres como "handlers"
    app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    #Los handlers son reglas que dicen: "Cuando ocurra X tipo de mensaje, ejecuta esta función"
    #CommandHandler es una función especual que detecta comandos de telegram tipo /help, /start, etc. 
    #Esta línea le dice al bot "Cuando un usuario escriba el comando /start (cuyo nombre es "start") llama a la función start"
    app.add_handler(CommandHandler("start", start))

    #El message handler maneja mensajes de texto normales 
    #Le dice al bot "cuando recibas un mensaje de texto que no sea un comando, llama a la función handle_message"
    #filters.TEXT acepta mensajes de tipo texto 
    #~filters.COMMAND excluye los comandos (mensajes que empiezan por /)
    #El operador & combina ambos filtros
    app.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), handle_message))

    print("Bot is running...")
    app.run_polling()

if __name__ == '__main__':
    main()


## Handler para mensajes de texto (núcleo del bot)
1. Extrae el user_id (identificador del chat) y el ltexto del usuario. Y envía un mensaje inmediato "Thinking". 
2. Si no existe un historial para ese user_id, crea la lista. 
3. Agrega el mensaje del usuario al historial como {"role": "user", "content": ...} — este formato es el típico para APIs de chat (sistema/user/assistant).
4. Llama a la API de Hugging Face via InferenceClient pidiendo una completación de chat. Parámetros: 
    - model: nombre del modelo 
    - messages: historial de conversación (contexto)
    - max_tokens: límite de tokens en la respuesta
5. Extrae el texto de respiesta del LLM (choices[0]...)
6. Añade la respuesta al historial para mantener contexto en futuros intercambios. 
7. Envía la respuesta al usuario vía reply_text
8. Por último captura cualquier excpeción de la llamada a la API o de la lógica y la notifica al usuario. 

Explicación: await update.message.reply_text("Thinking...")
- update: es un objeto de tipo de telegram.Update que representa un evento entrante que el bot recibe desde telegram. Ejemplo de lo que puede contener: 
    update = {
        "update_id": 123456,
        "message": {
            "message_id": 42,
            "from": {"id": 987654321, "first_name": "Alice"},
            "chat": {"id": 987654321, "type": "private"},
            "date": 1698163837,
            "text": "Hola bot!"
        }
    }
- update.message: es el mensaje concreto que envió el usuario
- reply_text() es un método del objeto Message cuyo propósito es responder a ese mensaje (en el mismo chat) enviando texto de vuelta al usuario. Internamente: 
    - Construye una petición HTTP a la API de telegram
    - Espera la respuesta (por eso es asíncrono)
    - Usa el TELEGRAM_TOKEN para autentificar
    - Envía el texto "Thinking"
    - Se pausa la ejecución de la función mientras se envía el mensaje y se recibe la confirmación

Qué devuelve? 
- reply_text() devuelve un objeto Message que representa el mensaje que el bot acaba de enviar.
- Esto te permitiría: Guardar el ID del mensaje. Editarlo después (por ejemplo, cambiar “Thinking…” por la respuesta final).
    ```ruby
    sent_msg = await update.message.reply_text("Thinking...")
    # unos segundos después
    await sent_msg.edit_text("Aquí está tu respuesta final ✨")
    ```

EN resumen: update.message.reply_text("Thinking...") → le dice al servidor de Telegram “envía este texto como respuesta al mensaje recibido”.


In [ ]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.message.chat_id
    user_message = update.message.text

    # Después de mandar el mensaje, el control vuelve al bucle principal mientras Telegram responde. Cuando responde, esta función se reanuda
    await update.message.reply_text('Thinking... 🤔')

    #guardar el contexto según el user_id permite que haya varios clientes a la vez guardando el contexto/historial por separado
    if user_id not in user_conversations:
        user_conversations[user_id] = []

    user_conversations[user_id].append({"role": "user", "content": user_message})

    try:
        #esta función no es asíncrona
        completion = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=user_conversations[user_id],
            max_tokens=500,
        )

        response_text = completion.choices[0].message["content"]
        user_conversations[user_id].append({"role": "assistant", "content": response_text})

        # Al terminar, responde al usuario
        await update.message.reply_text(response_text)

    except Exception as e:
        print(f"Error: {e}")
        await update.message.reply_text('Something went wrong! 😥')